In [2]:
!pip install pyarrow
!pip install fastparquet
!pip install scikit-learn
!pip install numpy
!pip install pandas
!pip install scipy

In [3]:
import datetime
import warnings

import numpy as np
import pandas as pd
from numpy import newaxis
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors


def set_up_printing():
    """Sets up display parameters"""

    max_dimensions_display_limit = 1000
    pd.set_option('display.max_columns', max_dimensions_display_limit)
    pd.set_option('display.max_colwidth', None)

    pd.set_option('display.max_rows', max_dimensions_display_limit)
    pd.set_option('display.width', None)

    pd.set_option('display.max_info_columns', max_dimensions_display_limit)
    pd.set_option('display.max_info_rows', max_dimensions_display_limit)

    warnings.filterwarnings('ignore')


def log_status(msg):
    print(datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S"), msg)


log_status('Пример рекоммендации вакансий на основе поиска ближайших пользователей')
'''важное замечание, ниже расчет на основе метода ближайших соседей только по тестовому набору, пока без валидации'''

# настроить отображение при выводе в консоль
set_up_printing()

# считать набор данных с векторами
user_vectors = pd.read_parquet(r'C:\Users\M_N_K\kosta_2023_dataton\KNN_users_vectors_min.parquet')

# как было сделано до этого при рассчете векторов, аналогично считать и обработать данные
dataframe = pd.read_parquet(r'C:\Users\M_N_K\kosta_2023_dataton\train_mfti.parquet')
dataframe = dataframe.drop_duplicates()
dataframe = dataframe[dataframe['event_type'].isin(['click_phone', 'click_response'])]
dataframe = dataframe.groupby('user_id').filter(lambda x: len(x) > 30)
dataframe['user_id'] = dataframe['user_id'].fillna(dataframe['cookie_id'])
dataframe = dataframe[['user_id', 'vacancy_id_']]

# выбрать случайным обрабом пользователя, для которого требуется найти рекоммендации
random_selection = np.random.choice(user_vectors.shape[0])
user_id_to_get_rec_for = user_vectors.index[random_selection]

# выбрать вакансии, с которыми уже взаимодействовал целевой пользователь
already_seen = dataframe[dataframe['user_id'] == user_id_to_get_rec_for]['vacancy_id_'].tolist()

# настроить параметры модели, без валидации можно использовать NearestNeighbors от sklearn
model = NearestNeighbors(metric="cosine", algorithm="brute", n_jobs=32)
model.fit(csr_matrix(user_vectors.values))

# выбрать ближайших соседей (пользователей), только тестовые данные, пока без валидации
request = np.array(user_vectors.iloc[random_selection, :].values)[:, newaxis].reshape(1, -1)
distance, suggestions = model.kneighbors(request, n_neighbors=30)

# вывести рекомендации вакансий на основе рекомендованных пользователей, с учетом порядка
top_limit = 5
recommendations_list = list()
for i in range(0, len(distance.flatten())):
    recommended_user_id = user_vectors.index[suggestions.flatten()[i]]
    
    # исключаем вакансии, с которыми целевой пользователь уже взаимодействовал
    rec_to_add_list_by_rec_user = [rec_to_add for rec_to_add in
                                   dataframe[dataframe['user_id'] == recommended_user_id]['vacancy_id_'].tolist() if
                                   rec_to_add not in already_seen]
    recommendations_list.extend(rec_to_add_list_by_rec_user)
    if len(recommendations_list) >= top_limit:
        break

log_status('Список рекомендаций для пользователя с id "{0}":\n'.format(user_vectors.index[random_selection]))
for rec in recommendations_list[0:top_limit]:
    print(rec)


30/04/2023 09:10:31 Пример рекоммендации вакансий на основе поиска ближайших пользователей
30/04/2023 09:10:48 Список рекомендаций для пользователя с id "31af9b292f4140798c271056435ef8d6":

240663
240663
134154
145456
140184
